In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Read the API key
API_KEY = os.getenv("AZURE_API_KEY")
ENDPOINT = os.getenv("AZURE_API_ENDPOINT")

In [ ]:
from openai import OpenAI
import os 

endpoint = ENDPOINT
model_name = "grok-4-fast-non-reasoning"
deployment_name = "grok-4-fast-non-reasoning"
api_key = API_KEY

client = OpenAI(
    base_url=f"{endpoint}",
    api_key=api_key
)

inputTest = """"
This is the attributes extracted from the database for you to enrich the properties

name: "Guarulhos Airport"
iata: "GRU"
country: "Brazil"
city: "Sao Paulo"
"""

promptTest = """"
# Identity

You are an assistant that enrichs entities from a database for analytics by identifying the corresponding real-world entity and adding new trustworthy factual attributes. These attributes will be transformed into property:value pairs in a property-graph database.

# Instructions

1. From the input node (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.
2. Add the most factual information you can confidently verify for the entity class. Prioritize core, verifiable facts (dates, identifiers, official names, locations, standardized codes, market share, estimated revenue).
3. Use consistent property names and information across entities of the same class. Try to make entities share the most amount of high-value information.

# Output format

Out a CSV with header `property,value`.
These will be related only to properties of nodes.
If no facts can be verified, output only the header row.
"""

folder_path = "../prompts/"

files = os.listdir(folder_path)
texts = []

for filename in files:
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
            texts.append(content)

for index,prompt in enumerate(texts):

    completion = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {
                "role": "system",
                "content": prompt,
                "role": "user",
                "content": inputTest,
            }
        ],
    )

    output_path = "../outputs/"+f"{model_name}_{files[index]}"
    with open(output_path, "w", encoding="utf-8") as outputFile:
        outputFile.write(completion.choices[0].message.content)
